In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
import string
import re
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
# from textstat.textstat import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
from keras.optimizers import Adam

/home/bhargav/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
embeddings_index = dict()
f = open('../glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [3]:
df = pd.read_csv("../Proper Experiments/Parsed Datasets/GAB.csv")
df.text=df.text.astype(str)
df.head(5)

,Unnamed: 0,class,text
0,0,1,joined gab remind retarded jew haters are typi...
1,1,0,left really scared of https : redd it 9rfkts
2,2,0,that literally looks like monkey supposed pre...
3,3,1,dumb cunt
4,4,0,makes asshole


In [4]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
vocabulary_size=len(tokenizer.word_index) + 1
print(vocabulary_size)

embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

40204


In [5]:
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    break

In [7]:
hidden_size=[75,100,125]
learning_rate=[0.0001,0.001,0.01]
score=[]
accuracy=[]

for i in learning_rate:
    sco=[]
    acc=[]
    for j in hidden_size:
        
        model_glove = Sequential()
        model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
        model_glove.add(LSTM(300))
        model_glove.add(Dense(j, activation='relu'))
        model_glove.add(Dense(1, activation='sigmoid'))
        model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=i), metrics=['accuracy'])
        
        model_glove.fit(Xtrain, np.array(ytrain), epochs = 3)

        pred=model_glove.predict_classes(X_val)
        
        acc.append(accuracy_score(y_val, pred))
        sco.append(f1_score(y_val, pred, average='macro'))
    score.append(sco)
    accuracy.append(acc)

Epoch 1/3
24317/24317 [==============================] - 189s 8ms/step - loss: 0.3935 - acc: 0.8411
Epoch 2/3
24317/24317 [==============================] - 202s 8ms/step - loss: 0.2842 - acc: 0.8932
Epoch 3/3
24317/24317 [==============================] - 192s 8ms/step - loss: 0.2623 - acc: 0.9029
Epoch 1/3
24317/24317 [==============================] - 182s 8ms/step - loss: 0.3919 - acc: 0.8387
Epoch 2/3
24317/24317 [==============================] - 192s 8ms/step - loss: 0.2843 - acc: 0.8941
Epoch 3/3
24317/24317 [==============================] - 190s 8ms/step - loss: 0.2623 - acc: 0.9031
Epoch 1/3
24317/24317 [==============================] - 202s 8ms/step - loss: 0.3895 - acc: 0.8420
Epoch 2/3
24317/24317 [==============================] - 182s 7ms/step - loss: 0.2816 - acc: 0.8947
Epoch 3/3
24317/24317 [==============================] - 184s 8ms/step - loss: 0.2598 - acc: 0.9045
Epoch 1/3
24317/24317 [==============================] - 194s 8ms/step - loss: 0.3053 - acc: 0.8859


In [10]:
print(score)

[[0.8989799621229537, 0.9053032225323522, 0.9026455462133729], [0.9100080436559107, 0.9167597481996144, 0.91467441833498], [0.9075613205105234, 0.9066145341753062, 0.9113086075184508]]


In [11]:
ac=[]
sc=[]
i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    model_glove = Sequential()
    model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
    model_glove.add(LSTM(300))
    model_glove.add(Dense(100, activation='relu'))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    
    print("Fold Number: "+str(i))  
    model_glove.fit(Xtrain, np.array(ytrain), epochs = 5)
    
    pred=model_glove.predict_classes(Xtest)
    
    a=accuracy_score(ytest, pred)
    s=f1_score(ytest, pred, average='macro')
    
    if i==0:
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_GAB.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_GAB.h5")
        print("Saved model to disk")
    elif s>max(sc):
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_GAB.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_GAB.h5")
        print("Saved model to disk")

    ac.append(a)
    sc.append(s)
    i=i+1    

Fold Number: 0
Epoch 1/5
24317/24317 [==============================] - 177s 7ms/step - loss: 0.3063 - acc: 0.8894
Epoch 2/5
24317/24317 [==============================] - 176s 7ms/step - loss: 0.2469 - acc: 0.9122
Epoch 3/5
24317/24317 [==============================] - 172s 7ms/step - loss: 0.2229 - acc: 0.9180
Epoch 4/5
24317/24317 [==============================] - 189s 8ms/step - loss: 0.1911 - acc: 0.9268
Epoch 5/5
24317/24317 [==============================] - 184s 8ms/step - loss: 0.1475 - acc: 0.9428
Saved model to disk
Fold Number: 1
Epoch 1/5
24318/24318 [==============================] - 179s 7ms/step - loss: 0.3043 - acc: 0.8868
Epoch 2/5
24318/24318 [==============================] - 173s 7ms/step - loss: 0.2508 - acc: 0.9106
Epoch 3/5
24318/24318 [==============================] - 173s 7ms/step - loss: 0.2431 - acc: 0.9119
Epoch 4/5
24318/24318 [==============================] - 164s 7ms/step - loss: 0.2187 - acc: 0.9199
Epoch 5/5
24318/24318 [===========================

In [12]:
print("Accuracies: ")
print(ac)
print("Mean: "+str(np.mean(np.array(ac))))
print("Std Dev: "+str(np.std(np.array(ac))))
print("Macro-F1-Scores: ")
print(sc)
print("Mean: "+str(np.mean(np.array(sc))))
print("Std Dev: "+str(np.std(np.array(sc))))

Accuracies: 
[0.9056076303239599, 0.9141447368421053, 0.9060700773153479, 0.8917585129133081, 0.9060700773153479]
Mean: 0.9047302069420138
Std Dev: 0.007228535069292236
Macro-F1-Scores: 
[0.903667438395628, 0.9120239286398569, 0.9037991224682413, 0.890391031753861, 0.9038682761453392]
Mean: 0.9027499594805853
Std Dev: 0.006956187766562902


In [5]:
# load json and create model
json_file = open('Models/model_GAB.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_glove = model_from_json(loaded_model_json)
# load weights into new model
model_glove.load_weights("Models/model_GAB.h5")
print("Loaded model from disk")

Loaded model from disk


In [14]:
# Dataset-1
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_GAB.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    GAB_model = model_from_json(loaded_model_json)
    # load weights into new model
    GAB_model.load_weights("Models/model_GAB.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in GAB_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    GAB_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    GAB_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=GAB_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0                                               text  class
0           0   ! ! ! mayasolovely : woman complain cleaning ...      0
1           1   ! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...      1
2           2   ! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...      1
3           3   ! ! ! ! ! ! ! ! ! c g anderson : viva based l...      1
4           4   ! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...      1
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f8ef9a3a9e8> False
<keras.layers.recurrent.LSTM object at 0x7f8ef9a1a048> True
<keras.layers.core.Dense object at 0x7f8ef9a1aef0> True
<keras.layers.core.Dense object at 0x7f8fbc2c62e8> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f8ef9a3a9e8> False
<keras.layers.recurrent.LSTM object at 0x7f8ef9a1a048> False
<keras.layers.core.Dense object at 0x7f8ef9a1aef0> True
<keras.layers.core.Dense object at 0x7f8fbc2c62e8> True
Epoch 1/8
4460/4460 [======

In [15]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.9184222321828777, 0.9168534289556253, 0.9047085201793722, 0.9141255605381166, 0.9091928251121076]
Mean: 0.9126605133936199
Std Dev: 0.005063267039162945
Macro-F1-Scores: 
[0.8532126475674864, 0.849986266428745, 0.8270131295711359, 0.8508475168248189, 0.8336981515111759]
Mean: 0.8429515423806724
Std Dev: 0.01055255684638451
Without Fine Tuning: 
Accuracies: 
[0.6033168982519049, 0.5941281936351412, 0.6060538116591928, 0.5975336322869955, 0.5948430493273542]
Mean: 0.5991751170321178
Std Dev: 0.004720243849362153
Macro-F1-Scores: 
[0.5710376927868304, 0.5628801924895848, 0.5718258617240717, 0.5661807503078331, 0.5632805659087766]
Mean: 0.5670410126434193
Std Dev: 0.003769924884557381


In [16]:
# Dataset-2
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Waseem_Hovy.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_GAB.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    GAB_model = model_from_json(loaded_model_json)
    # load weights into new model
    GAB_model.load_weights("Models/model_GAB.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in GAB_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    GAB_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    GAB_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=GAB_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0                                               text  class
0           0   mralmubarak yes tried murder way heaven total...      1
1           1   nigzzzk command fear allah quran ritual grove...      1
2           2   ib alkisai islamic documentation based belief...      1
3           3   benfrancisallen : good weeks isis new front o...      0
4           4  self described radical feminist makes death th...      1
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f8eef521eb8> False
<keras.layers.recurrent.LSTM object at 0x7f8eef521d30> True
<keras.layers.core.Dense object at 0x7f8eef521cf8> True
<keras.layers.core.Dense object at 0x7f8ef99c2710> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f8eef521eb8> False
<keras.layers.recurrent.LSTM object at 0x7f8eef521d30> False
<keras.layers.core.Dense object at 0x7f8eef521cf8> True
<keras.layers.core.Dense object at 0x7f8ef99c2710> True
Epoch 1/8
2841/2841 [======

In [17]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.7812170242701372, 0.7821956368754398, 0.7814919071076707, 0.7748064743138635, 0.7849348820837733]
Mean: 0.7809291849301769
Std Dev: 0.0033327647154210867
Macro-F1-Scores: 
[0.7303291658431725, 0.7198141712203021, 0.7139484032148189, 0.7121427739429451, 0.7224493326619954]
Mean: 0.7197367693766469
Std Dev: 0.006492099860553065
Without Fine Tuning: 
Accuracies: 
[0.6950404502286317, 0.6980999296270233, 0.6921182266009852, 0.6970443349753694, 0.6948257655755016]
Mean: 0.6954257414015022
Std Dev: 0.002060058361221637
Macro-F1-Scores: 
[0.45261176752754007, 0.46585126069858895, 0.4418764890120311, 0.45908417793334233, 0.4497562250570484]
Mean: 0.45383598404571013
Std Dev: 0.008161900184110506


In [18]:
# Dataset-3
df=pd.read_csv('../Proper Experiments/Parsed Datasets/OffensEval_2019.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_GAB.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    GAB_model = model_from_json(loaded_model_json)
    # load weights into new model
    GAB_model.load_weights("Models/model_GAB.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in GAB_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    GAB_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    GAB_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=GAB_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0                                               text  class
0           0                 user ask native americans take is       1
1           1   user user home you re drunk ! ! ! user maga t...      1
2           2  amazon investigating chinese employees selling...      0
3           3   user someone should have taken piece shit vol...      1
4           4   user user obama wanted liberals amp; illegals...      0
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f8eed8c4fd0> False
<keras.layers.recurrent.LSTM object at 0x7f8eee7b74a8> True
<keras.layers.core.Dense object at 0x7f8eefa36240> True
<keras.layers.core.Dense object at 0x7f8eede2f278> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f8eed8c4fd0> False
<keras.layers.recurrent.LSTM object at 0x7f8eee7b74a8> False
<keras.layers.core.Dense object at 0x7f8eefa36240> True
<keras.layers.core.Dense object at 0x7f8eede2f278> True
Epoch 1/8
2383/2383 [======

In [19]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.7470637583892618, 0.7519932857742342, 0.7356273604699958, 0.7486361728913135, 0.7473772555602182]
Mean: 0.7461395666170046
Std Dev: 0.005538800471038945
Macro-F1-Scores: 
[0.6934495687871085, 0.6941948276726873, 0.6726836759788813, 0.6793060025499744, 0.6857924718069486]
Mean: 0.6850853093591199
Std Dev: 0.008254030968001232
Without Fine Tuning: 
Accuracies: 
[0.6740771812080537, 0.6785564414603441, 0.6785564414603441, 0.6802349979018044, 0.6772975241292488]
Mean: 0.6777445172319589
Std Dev: 0.0020576730652975515
Macro-F1-Scores: 
[0.4294674496070668, 0.4433614857859389, 0.4401821224143303, 0.44416507493388185, 0.43852498320195676]
Mean: 0.43914022318863494
Std Dev: 0.0052553746959679995


In [20]:
# Dataset-4
df=pd.read_csv('../Proper Experiments/Parsed Datasets/hatEval_2019.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_GAB.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    GAB_model = model_from_json(loaded_model_json)
    # load weights into new model
    GAB_model.load_weights("Models/model_GAB.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in GAB_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    GAB_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    GAB_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=GAB_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0                                               text  class
0           0  hurray saving many ways potus realdonaldtrump ...      0
1           1  would young fighting age men vast majority one...      0
2           2   kamalaharris illegals dump kids border like r...      0
3           3  times : nearly white states pose an array prob...      0
4           4  orban brussels : european leaders ignoring peo...      0
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f8eda4e1dd8> False
<keras.layers.recurrent.LSTM object at 0x7f8eda9e53c8> True
<keras.layers.core.Dense object at 0x7f8eda4e1470> True
<keras.layers.core.Dense object at 0x7f8eeeea19e8> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f8eda4e1dd8> False
<keras.layers.recurrent.LSTM object at 0x7f8eda9e53c8> False
<keras.layers.core.Dense object at 0x7f8eda4e1470> True
<keras.layers.core.Dense object at 0x7f8eeeea19e8> True
Epoch 1/8
1618/1618 [======

In [21]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8344657195799876, 0.8263288009888752, 0.8244746600741656, 0.830655129789864, 0.830655129789864]
Mean: 0.8293158880445513
Std Dev: 0.003534630930877607
Macro-F1-Scores: 
[0.5619164162708798, 0.5640440732995238, 0.5259185282089164, 0.552075881941666, 0.5248928326474622]
Mean: 0.5457695464736897
Std Dev: 0.017113443302391007
Without Fine Tuning: 
Accuracies: 
[0.7449042618900555, 0.7311495673671199, 0.7367119901112484, 0.723114956736712, 0.7169344870210136]
Mean: 0.7305630526252298
Std Dev: 0.009849219742902727
Macro-F1-Scores: 
[0.5444206352180474, 0.5283282502447724, 0.5426248566696394, 0.5332191383862451, 0.510998901957472]
Mean: 0.5319183564952353
Std Dev: 0.012026143137536107


In [22]:
# Dataset-5
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_1.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_GAB.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    GAB_model = model_from_json(loaded_model_json)
    # load weights into new model
    GAB_model.load_weights("Models/model_GAB.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in GAB_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    GAB_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    GAB_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=GAB_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0                                               text  class
0           0   ! ! ! mayasolovely : woman complain cleaning ...      0
1           1   ! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...      0
2           2   ! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...      0
3           3   ! ! ! ! ! ! ! ! ! c g anderson : viva based l...      0
4           4   ! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...      0
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f8ed91cb048> False
<keras.layers.recurrent.LSTM object at 0x7f8ed91cb780> True
<keras.layers.core.Dense object at 0x7f8ed91cb860> True
<keras.layers.core.Dense object at 0x7f8ed8496710> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f8ed91cb048> False
<keras.layers.recurrent.LSTM object at 0x7f8ed91cb780> False
<keras.layers.core.Dense object at 0x7f8ed91cb860> True
<keras.layers.core.Dense object at 0x7f8ed8496710> True
Epoch 1/8
4460/4460 [======

In [23]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.9401613626176603, 0.9457642312864186, 0.9428251121076233, 0.9405829596412556, 0.9349775784753364]
Mean: 0.9408622488256588
Std Dev: 0.0035519249250605704
Macro-F1-Scores: 
[0.6186489587984769, 0.5956305189601324, 0.5249593690518108, 0.6490144400311816, 0.6054089542330521]
Mean: 0.5987324482149308
Std Dev: 0.04104049464066866
Without Fine Tuning: 
Accuracies: 
[0.575302554908113, 0.5941281936351412, 0.5800448430493274, 0.5733183856502242, 0.5856502242152466]
Mean: 0.5816888402916105
Std Dev: 0.007536338010350465
Macro-F1-Scores: 
[0.43508478080989654, 0.45019481396482564, 0.4428803984679175, 0.43759107591690183, 0.441474071285161]
Mean: 0.4414450280889405
Std Dev: 0.005175433842701071


In [24]:
# Dataset-6
df=pd.read_csv('../Proper Experiments/Parsed Datasets/StormfrontWS.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_GAB.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    GAB_model = model_from_json(loaded_model_json)
    # load weights into new model
    GAB_model.load_weights("Models/model_GAB.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in GAB_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    GAB_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    GAB_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=GAB_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      0  march 13th 2014 booklet downloaded 18 300 time...
1           1      0  order help increase booklets downloads would g...
2           2      0  simply copy paste following text youtube video...
3           3      1  click free download colorfully illustrated 132...
4           4      0              click download 7 42 green banner link
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f8eb60f9400> False
<keras.layers.recurrent.LSTM object at 0x7f8eb6111160> True
<keras.layers.core.Dense object at 0x7f8eb60c3b70> True
<keras.layers.core.Dense object at 0x7f8ed9f4c978> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f8eb60f9400> False
<keras.layers.recurrent.LSTM object at 0x7f8eb6111160> False
<keras.layers.core.Dense object at 0x7f8eb60c3b70> True
<keras.layers.core.Dense object at 0x7f8ed9f4c978> True
Epoch 1/8
1926/1926 [======

In [25]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.895746887966805, 0.8878504672897196, 0.893042575285566, 0.8966770508826584, 0.893042575285566]
Mean: 0.893271911342063
Std Dev: 0.0030730353154578005
Macro-F1-Scores: 
[0.6356254519093257, 0.6071955977384401, 0.6343518794996849, 0.6077300737820436, 0.6183276897029397]
Mean: 0.6206461385264868
Std Dev: 0.012372098819251817
Without Fine Tuning: 
Accuracies: 
[0.8884854771784232, 0.8904465212876428, 0.8920041536863966, 0.8909657320872274, 0.8811007268951194]
Mean: 0.8886005222269618
Std Dev: 0.0039203776019538
Macro-F1-Scores: 
[0.607842024251128, 0.5976483796967804, 0.5941056317478844, 0.6084532130341342, 0.5604628427297397]
Mean: 0.5937024182919333
Std Dev: 0.017539195980786692


In [26]:
# Dataset-7
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Reddit.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_GAB.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    GAB_model = model_from_json(loaded_model_json)
    # load weights into new model
    GAB_model.load_weights("Models/model_GAB.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in GAB_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    GAB_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    GAB_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=GAB_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      1  subsection retarded hungarians ohh boy brace l...
1           1      0   hiii got work 444 mainly typa guys imagine wr...
2           2      0                    wow guess soyboys every country
3           3      0   owen benjamin soyboy song goes every country ...
4           4      0   y all hear sumn means live small town for wor...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f8eb5b81ba8> False
<keras.layers.recurrent.LSTM object at 0x7f8eb43e8a58> True
<keras.layers.core.Dense object at 0x7f8eb43e8eb8> True
<keras.layers.core.Dense object at 0x7f8eb42795c0> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f8eb5b81ba8> False
<keras.layers.recurrent.LSTM object at 0x7f8eb43e8a58> False
<keras.layers.core.Dense object at 0x7f8eb43e8eb8> True
<keras.layers.core.Dense object at 0x7f8eb42795c0> True
Epoch 1/8
4018/4018 [======

In [27]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.9121672057725803, 0.8944748631159781, 0.9066699850671976, 0.9096565455450473, 0.9039323046291687]
Mean: 0.9053801808259945
Std Dev: 0.006116157251972741
Macro-F1-Scores: 
[0.8766773368451477, 0.8514716549492245, 0.8687847811769058, 0.8727921536784975, 0.8648849819892873]
Mean: 0.8669221817278127
Std Dev: 0.008671307390277138
Without Fine Tuning: 
Accuracies: 
[0.9027121174421497, 0.8904927824788452, 0.9009457441513191, 0.9036834245893479, 0.9009457441513191]
Mean: 0.8997559625625963
Std Dev: 0.004749767896754823
Macro-F1-Scores: 
[0.864413519915892, 0.8480247655194539, 0.863228415740239, 0.8668642769352086, 0.8636214471870863]
Mean: 0.8612304850595761
Std Dev: 0.006722659849594528


In [6]:
# Dataset-8
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_2.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_GAB.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    GAB_model = model_from_json(loaded_model_json)
    # load weights into new model
    GAB_model.load_weights("Models/model_GAB.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in GAB_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in GAB_model.layers:
        print(layer, layer.trainable)
    GAB_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    GAB_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=GAB_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      0   ! ! ! mayasolovely : woman complain cleaning ...
1           1      0              momma said pussy cats inside doghouse
2           2      0   addicted2guys : - simplyaddictedtoguys http :...
3           3      0   allaboutmanfeet : http : t co 3gzupfumev woof...
4           4      0   allyhaaaaa : lemmie eat oreo amp; dishes one ...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f00c24d2470> False
<keras.layers.recurrent.LSTM object at 0x7f00c24d2588> True
<keras.layers.core.Dense object at 0x7f00c24d2748> True
<keras.layers.core.Dense object at 0x7f00c247fb70> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f00c24d2470> False
<keras.layers.recurrent.LSTM object at 0x7f00c24d2588> False
<keras.layers.core.Dense object at 0x7f00c24d2748> True
<keras.layers.core.Dense object at 0x7f00c247fb70> True
Epoch 1/8
1006/1006 [======

In [7]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8988095238095238, 0.9175769612711023, 0.9204771371769384, 0.915506958250497, 0.904572564612326]
Mean: 0.9113886290240775
Std Dev: 0.008277073083226007
Macro-F1-Scores: 
[0.860546875, 0.8859972529458542, 0.8876134618070103, 0.8828750470842037, 0.8671362535769315]
Mean: 0.8768337780827998
Std Dev: 0.010917627741204505
Without Fine Tuning: 
Accuracies: 
[0.8958333333333334, 0.9036742800397219, 0.9085487077534792, 0.9085487077534792, 0.8926441351888668]
Mean: 0.901849832813776
Std Dev: 0.006542500949258692
Macro-F1-Scores: 
[0.8528035336052704, 0.8627234385825854, 0.8691533820044901, 0.8687429090083958, 0.8478148812191841]
Mean: 0.8602476288839851
Std Dev: 0.008574376078928095
